Inspired by https://perso.ensta-paris.fr/~becache/COURS-ONDES/Poly-num-0209.pdf

$$
\frac{u_{ij}^{n+1} - 2u_{ij}^{n} + u_{ij}^{n-1}}{\Delta t^2} - \frac{u_{i+1j}^{n} - 2u_{ij}^{n} + u_{i-1j}^{n}}{\Delta x^2} - \frac{u_{ij+1}^{n} - 2u_{ij}^{n} + u_{ij-1}^{n}}{\Delta x^2} = g(n\Delta t) f(x_i, y_j)
$$

Rearranging for explicit terms, we get:

In [ ]:
import holoviews as hv
hv.extension('bokeh', logo=False)

import numba
import numpy as np

@numba.jit
def step(U, V, W, dt, dx, dy, g, t, f):
    """U = u^n+1, V = u^n, W = u^n-1."""
    N, M = U.shape
    for i in range(1, N-1):
        for j in range(1, M-1):
            U[i, j] = dt**2 / dx ** 2 * (V[i+1, j] - 2 * V[i, j] + V[i-1, j]) + \
                      dt**2 / dy ** 2 * (V[i, j+1] - 2 * V[i, j] + V[i, j-1]) + \
                      dt**2 * g(t) * f(i * dx, j * dy) + \
                      2 * V[i, j] - W[i, j]            
    return U



@numba.jit
def g(t):
    return np.sin(2 * np.pi * 15e-2 * t)

@numba.jit
def f(x, y):
    x0, y0 = (62.5, 32.5)
    return np.exp(- np.abs((x - x0)**2 + (y - y0)**2))

def n_steps(U, V, W, dt, dx, dy, g, t0, f, nsteps=100):
    for i in range(nsteps):
        t = t0 + i * dt
        U = step(U, V, W, dt, dx, dy, g, t, f)
        W = V.copy()
        V = U.copy()
    return U, V, W, t

In [ ]:
N, M = 250, 260
U = np.zeros((N, M))
V = np.zeros((N, M))
W = np.zeros((N, M))
dt = 0.1
dx = 0.5
dy = dx
t = 0

In [ ]:
N * dx / 2, M * dy / 4

In [ ]:
snapshots = []
for _ in range(30):
    U, V, W, t = n_steps(U, V, W, dt, dx, dy, g, t, f, nsteps=50)
    snapshots.append(U.copy())

In [ ]:
%%output holomap='scrubber'
hv.HoloMap({ind: hv.Image(U.T).opts(cmap='seismic', tools=['hover'], colorbar=True, width=700, height=600).redim.range(z=(-1e-1, 1e-1)) for ind, U in enumerate(snapshots)})